In [1]:
import os
import datetime
import time
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util
from utils.detect_utils import detect_objects_no_vis

In [54]:
# Path to frozen detection graph
EXPORT_NAME = 'ssd_mobilenetv2_1_160_300000'
MODEL_NAME = 'ssd_mobilenetv2_1_160'
CWD_PATH = os.getcwd()
PATH_TO_CKPT = os.path.join(CWD_PATH, '..', 'training/models', MODEL_NAME, 'export', EXPORT_NAME, 'frozen_inference_graph.pb')
# Path to labels
PATH_TO_LABELS = os.path.join(CWD_PATH, '..', 'training/data/label_map.pbtxt')
# Number of classes
NUM_CLASSES = 21
# Load label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
# Path to images
PATH_TO_IMAGES = os.path.join(CWD_PATH, '..', 'training/data/sample_images')

In [55]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    sess = tf.Session(graph=detection_graph)

In [56]:
from IPython.display import display, clear_output

execution_times = []
# run 100 different images and take average speed
i = 1
for jpgfile in os.listdir(PATH_TO_IMAGES):
    #print(os.path.join(PATH_TO_IMAGES, jpgfile))
    clear_output(wait=True)
    display(i)
    i += 1
    if jpgfile == '000000000086.jpg':
        continue
    start_time = datetime.datetime.now()
    image = cv2.imread(os.path.join(PATH_TO_IMAGES, jpgfile), -1)
    boxes, scores, classes, num_detections = detect_objects_no_vis(image, sess, detection_graph)
    end_time = datetime.datetime.now()
    execution_times.append((end_time - start_time).total_seconds())

100

In [57]:
execution_times = np.array(execution_times, dtype=np.float)
print("\n ----- SUMMARY ----- \n")
print("Average runtime: %ss" % np.average(execution_times))
print("Std Dev: %ss" % np.std(execution_times))
print("95th percentile: %ss" % np.percentile(execution_times,95))
print("99th percentile: %ss" % np.percentile(execution_times,99))


 ----- SUMMARY ----- 

Average runtime: 0.05143777777777778s
Std Dev: 0.18293476709204656s
95th percentile: 0.0401721s
99th percentile: 0.07838393999999275s
